In [ ]:
!pip install requests
!pip install beautifulsoup4
!pip install janome
!pip install scikit-learn
!pip install pandas
!pip list

In [ ]:
!pip install jellyfish

In [ ]:
!pip install Document

In [ ]:
!pip install docx

In [ ]:
!pip uninstall python-docx
!pip install python-docx

In [ ]:
from tf_idf import doc1,doc2

In [ ]:
print(doc1)

In [ ]:
from collections import defaultdict

In [ ]:
Doc = defaultdict(dict)
Doc[0]["doc"] = str(doc1)
Doc[1]["doc"] = str(doc2)


In [ ]:
print(Doc)

In [ ]:
print(Doc[0]["doc"])
print(Doc[1]["doc"])

In [ ]:
for i in Doc.keys():
    tmp = [t.replace(' ','').lower() for t in Doc[i]["doc"]]
    Doc[i]["doc"] = "".join(tmp)
print("len(Doc[0][doc]): {}".format(len(Doc[0]["doc"])))
print(Doc[0]["doc"])
print("len(Doc[1][doc]): {}".format(len(Doc[1]["doc"])))
print(Doc[1]["doc"])

In [ ]:
from janome.tokenizer import Tokenizer
from janome.analyzer import Analyzer
from janome.tokenfilter import POSStopFilter

tokenizer = Tokenizer()
token_filters = [POSStopFilter(['記号','助詞','助動詞','動詞'])]
a = Analyzer(tokenizer=tokenizer, token_filters=token_filters)

In [ ]:
test_tokens = a.analyze(Doc[0]["doc"])
for t in test_tokens:
    print(t)

In [ ]:
for i in Doc:
    texts_flat = list(tokenizer.tokenize(Doc[i]["doc"], wakati=True))
    tokens = a.analyze(' '.join(texts_flat))  # `a.analyze` が文字列を受け取る場合
    Doc[i]["wakati"] = ' '.join([t.surface for t in tokens])

In [ ]:
print("Doc[0][wakati]: {}".format(Doc[0]["wakati"]))

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import random
vectorizer = CountVectorizer()

In [ ]:
X = vectorizer.fit_transform([Doc[i]["wakati"] for i in Doc.keys()])
for i, bow in enumerate(X.toarray()):
    Doc[i]["bow"] = bow

In [ ]:
WORDS = vectorizer.get_feature_names_out()
print(WORDS)

In [ ]:
n = 3
for w_idx , count in enumerate(Doc[0]["bow"]):
    if count >= n:
        print(" - {}\t{}:Words[{}]".format(count, WORDS[w_idx],w_idx))

In [ ]:
def calc_tf(b_idx, w_idx):
    """b_idx 番目の WORD[w_idx] の TF値を算出する"""
    # WORD[w_idx] の出現回数の和
    word_count = Doc[b_idx]["bow"][w_idx]
    if word_count == 0:
        return 0.0
    # 全単語の出現回数の和
    sum_of_words = sum(Doc[b_idx]["bow"])
    # TF値計計算
    return word_count/float(sum_of_words)

In [ ]:
index = 0
sample_tfs = [calc_tf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfs_sorted = sorted(enumerate(sample_tfs), key=lambda x:x[1], reverse=True)
for i, tf in tfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tf, 4)))

In [ ]:
import math

def calc_idf(w_idx):
    """WORD[w_idx] の IDF値を算出する"""
    # 総文書数
    N = len(Doc.keys())
    print(f"N:{N}")
    # 単語 word が出現する文書数 df を計算
    df = len([i for i in Doc.keys() if Doc[i]["bow"][w_idx] > 0])
    # idf を計算
    return math.log2(N/float(df + 1))

In [ ]:
IDFS = [calc_idf(w_idx) for w_idx, word in enumerate(WORDS)]

In [ ]:
print(IDFS)

In [ ]:
idfs_sorted  = sorted(enumerate(IDFS), key=lambda x:x[1], reverse=True)
print("# IDF values")
for w_idx, idf in idfs_sorted[:10]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))
print("︙")
for w_idx, idf in idfs_sorted[-10:]:
    print("{}\t{}".format(WORDS[w_idx], round(idf, 4)))

In [ ]:
def calc_tfidf(b_idx, w_idx):
    """b_idx 番目の WORD[w_idx] の TF-IDF値を算出する"""
    return calc_tf(b_idx, w_idx) * calc_idf(w_idx)

In [ ]:
index = 0
sample_tfs = [calc_tfidf(index, w_idx) for w_idx, word in enumerate(WORDS)]
tfs_sorted = sorted(enumerate(sample_tfs), key=lambda x:x[1], reverse=True)
for i, tf in tfs_sorted[:20]:
    print("{}\t{}".format(WORDS[i], round(tf, 4)))
